In [1]:
import csv
import os
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

# Integridad de datos

## Préstamos

In [2]:
%%time

dtypes = {
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
}

prestamos_df = pd.read_csv('ultimos_prestamos.csv', header=0, dtype=dtypes)
prestamos_df = prestamos_df.fillna('')

CPU times: user 284 ms, sys: 16 ms, total: 300 ms
Wall time: 298 ms


In [3]:
prestamos_df.shape

(156392, 6)

In [4]:
prestamos_df.head()

,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA
0,,,,ARM,88429527,CTA
1,,,,ARM,88429529,CTA
2,docente,postgrado,psicología,ARM,88209238,CTF
3,estudiante,,,ARM,88439347,CTB
4,,,,ARM,88429493,CTA


## Socios activos 2016

In [5]:
# 1.       CTA: categoría para niños y jóvenes hasta bachillerato
# 2.       CTI: categoría infantil, hasta doce años y solo es para Bogotá (blaa –biblioteca Luis Ángel Arango)
# 3.       CTB: público general (solo permite préstamo de libros)
# 4.       CTC: público en general (permite préstamo de libros y material audiovisual)
# 5.       CTF: categoría familiar, pueden acceder a los beneficios hasta tres personas del grupo familiar que se asocia
# 6.       CE1, CE2, CE2: categorías empresariales (solo para instituciones)
# 7.       CIS: categoría Inclusión Social
# 8.       CTU: categoría Universitaria
# 9.       VIP: Investigadores, préstamo de 30 materiales (libros y material audiovisual)

#      Categoría de lector: CE1 Categoría empresarial                                   218
#      Categoría de lector: CE2 Categoría empresarial 2                                   5
#      Categoría de lector: CEM Categoría Empleado/Pensionado                          1593
#      Categoría de lector: CIN Categoría Interbibliotecario                            128
#      Categoría de lector: CIP Categoría Investigador                                  351
#      Categoría de lector: CIS Categoría Inclusión Preferencial                      15974
#      Categoría de lector: CIV Categoría Investigadores                                 72
#      Categoría de lector: CML Categoría Mediador de Proyectos Culturales                2
#      Categoría de lector: CSU Categoría Sucursal                                       91
#      Categoría de lector: CTA Categoría Básica A                                    10610
#      Categoría de lector: CTB Categoría B                                            1359
#      Categoría de lector: CTC Categoría C                                             224
#      Categoría de lector: CTF Categoría familiar                                     1105
#      Categoría de lector: CTI Categoría Infantil                                     1004
#      Categoría de lector: CTU Categoría General                                     17402
#      Categoría de lector: PTE PROCESOS TÉCNICOS                                        95

In [6]:
%%time

dtypes = {
    'CODBAR': str,
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Fech. Nacim': pd.tslib.Timestamp,
    'Edad': float,
}

perfiles_df = pd.read_csv('socios_activos_2016.csv', header=0, dtype=dtypes, parse_dates=['Fech. Nacim'])
perfiles_df = perfiles_df.fillna('')

CPU times: user 512 ms, sys: 36 ms, total: 548 ms
Wall time: 782 ms


In [7]:
perfiles_df.shape

(50272, 11)

In [8]:
perfiles_df.head()

,CODBAR,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Fech. Nacim,Edad
0,88180782,BLAA,,chapinero,bogota,CIN,universidad,,,,
1,88183608,BLAA,sra.,ayacuyo ii,bogota,CEM,pensionado,,,1957-07-30 00:00:00,59
2,88181732,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,,
3,88182347,ARM,sr.,montenegro,montenegro,CEM,empleado,profesional,economía,1963-09-20 00:00:00,53
4,88181284,BLAA,sr.,la soledad,bogota,CEM,,,,,


## Columnas comunes

In [9]:
used_colums_1 = [
    # Ejemplar
    'TITULO', 'AUTOR', 'SUCURSAL', 'COD_BARRAS_LIBRO', 'DEWEY', 'DESC_LOCALIZACION_EJEMPLAR',
    'FECHA_PUB','PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION','NIVEL_EDUCACION','AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO','SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO','FECHA_REAL_DEVOLUCION'
]

In [10]:
used_colums_2 = [
    # Identificación
    'CODBAR',
    # Localización (I)
    'Sucursal',
    # Género
    'Tratamiento',
    # Localización (II)
    'Barrio', 'Ciudad',
    # Educación/Ocupación
    'CATEG', 'Ocupación1', 'Nivel Estudio1', 'Área de Estudio1',
    # Edad
    'Fech. Nacim',
]

In [11]:
mappings = {
    'OCUPACION': 'Ocupación1',
    'NIVEL_EDUCACION': 'Nivel Estudio1',
    'AREA_ESTUDIO': 'Área de Estudio1',
    'SUCURSAL_DEL_LECTOR': 'Sucursal',
    'COD_BARRAS_LECTOR': 'CODBAR',
    'CATEGORÍA': 'CATEG',
}

common_columns = ['OCUPACION', 'NIVEL_EDUCACION', 'AREA_ESTUDIO', 'SUCURSAL_DEL_LECTOR', 'CATEGORÍA']

# Comprobación

In [12]:
perfiles_df.set_index('CODBAR', inplace=True, verify_integrity=True)

In [13]:
def users_equals(s1, s2):
    return all(map(lambda x: s1[x] == s2[mappings[x]], common_columns))

In [30]:
bad = {}
for i1, data1 in prestamos_df.iterrows():
    codbar = data1['COD_BARRAS_LECTOR']
    if codbar in perfiles_df.index:
        i2 = codbar
        data2 = perfiles_df.ix[i2]
        if not users_equals(data1, data2):
            bad[codbar] = (data1, data2)

In [31]:
len(prestamos_df)

156392

In [32]:
len(perfiles_df)

50272

In [33]:
len(bad)

1154

In [34]:
len(bad)

1154

In [38]:
for k in bad:
    print(bad[k][0])
    print()
    print(bad[k][1])
    break

OCUPACION                 empleado
NIVEL_EDUCACION        profesional
AREA_ESTUDIO              economía
SUCURSAL_DEL_LECTOR           BLAA
COD_BARRAS_LECTOR         88439180
CATEGORÍA                      CTB
Name: 135552, dtype: object

Sucursal                           BLAA
Tratamiento                         sr.
Barrio                         cedritos
Ciudad                           bogota
CATEG                               CTU
Ocupación1                independiente
Nivel Estudio1              profesional
Área de Estudio1                derecho
Fech. Nacim         1983-07-14 00:00:00
Edad                                 33
Name: 88439180, dtype: object


In [26]:
len(bad2)

0

In [22]:
bad2[0][0]

IndexError: list index out of range

In [ ]:
bad2[0][1]

In [ ]:
bad2[1][0]

In [ ]:
bad2[1][1]

In [ ]:
pd.Series(bad).count()

In [ ]:
perfiles_df.ix['88180782']

In [ ]:
'88180782' in perfiles_df.index

In [ ]:
for x, y in prestamos_aux_df.iterrows():
    print(x, y)
    break

In [ ]:
perfiles_df['CODBAR'].count()

In [ ]:
perfiles_df['CODBAR'].nunique()